### purpose

train the first 225 levels of GF with the addition of nuisance envs

since there are three nuissance envs, train the 225 levels with either 1, 2, or all nuissance envs to look at effect of additional envs (full-factorial)

### Notes

I ended up not doing a full factorial. I only ran "ISO-PSsd" and "ISO-TSsd-PSsd"

In [1]:
from pythonimports import *

import MVP_summary_functions as mvp
import MVP_06_validate_RONA as mvp06

# canonical dirs
gfc_training_dir = '/home/b.lind/offsets/run_20220919_0-225/gradient_forests/training/training_files'
gfc_training_shdir = '/home/b.lind/offsets/run_20220919_0-225/gradient_forests/training/training_shfiles'

# re: nuisance
maindir = makedir('/home/b.lind/offsets/run_20220919_nuisance')  # upper dir of all nuis outerdirs
nuis_dir = '/home/b.lind/offsets/run_20220919_tutorial/tutorial/'

t1 = dt.now()  # notebook timer

mvp.latest_commit()
session_info.show()

#########################################################
Today:	April 11, 2023 - 11:19:11
python version: 3.8.5

Current commit of pythonimports:
commit 4ecd56c8c80ec4876790fcd425cb75b9db9a4f24  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Tue Mar 21 13:20:54 2023 -0400

Current commit of MVP_offsets:
commit ef86c4b94658027c580e1bbfdfc1195b0b7077e9  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Thu Mar 23 10:58:00 2023 -0400
#########################################################



# load pop data

just a file to get lat/long for pops

In [2]:
_, locations, _ = mvp06.get_pop_data(mvp.slimdir, '1231094')
locations = locations[['lat', 'lon']]  # rearrange here to preserve canonical rangefile column order


Reading in info for subsampled individuals ...


In [3]:
locations

,lat,lon
subpopID,,
1,1.0,1.0
2,1.0,2.0
3,1.0,3.0
4,1.0,4.0
5,1.0,5.0
...,...,...
96,10.0,6.0
97,10.0,7.0
98,10.0,8.0


# get env data (temp and sal)

In [4]:
# env data used previously
efiles = fs(gfc_training_dir, pattern='envfile_GFready_pooled')

len(efiles)  # num seeds

225

In [5]:
# map files to seeds
envfiles = {}
for f in efiles:
    seed = op.basename(f).split("_")[0]
    envfiles[seed] = f

# get nuisance env data

each combo of nuisance envs will be the 'outerdir' (just the way the code works)

In [6]:
# get nuis env files (files created in 00_create_nuisance_env_files.ipynb)
_files = fs(nuis_dir, startswith='nuisance')

nuis_files = {}
for f in _files:
    envs = op.basename(f).split("_")[1]  # which factorial combo of the 3 nuisance envs are being used
    nuis_files[envs] = f

print(len(nuis_files))

nuis_files

7


{'ISO-PSsd': '/home/b.lind/offsets/run_20220919_tutorial/tutorial/nuisance_ISO-PSsd_vals.txt',
 'ISO-TSsd-PSsd': '/home/b.lind/offsets/run_20220919_tutorial/tutorial/nuisance_ISO-TSsd-PSsd_vals.txt',
 'ISO-TSsd': '/home/b.lind/offsets/run_20220919_tutorial/tutorial/nuisance_ISO-TSsd_vals.txt',
 'ISO': '/home/b.lind/offsets/run_20220919_tutorial/tutorial/nuisance_ISO_vals.txt',
 'PSsd': '/home/b.lind/offsets/run_20220919_tutorial/tutorial/nuisance_PSsd_vals.txt',
 'TSsd-PSsd': '/home/b.lind/offsets/run_20220919_tutorial/tutorial/nuisance_TSsd-PSsd_vals.txt',
 'TSsd': '/home/b.lind/offsets/run_20220919_tutorial/tutorial/nuisance_TSsd_vals.txt'}

### create outerdirs

In [7]:
outerdirs = {}
for envs, f in nuis_files.items():
    outerdir = makedir(op.join(maindir, f'{envs}_0-225'))
    
    outerdirs[envs] = outerdir
    
outerdirs

{'ISO-PSsd': '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225',
 'ISO-TSsd-PSsd': '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225',
 'ISO-TSsd': '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd_0-225',
 'ISO': '/home/b.lind/offsets/run_20220919_nuisance/ISO_0-225',
 'PSsd': '/home/b.lind/offsets/run_20220919_nuisance/PSsd_0-225',
 'TSsd-PSsd': '/home/b.lind/offsets/run_20220919_nuisance/TSsd-PSsd_0-225',
 'TSsd': '/home/b.lind/offsets/run_20220919_nuisance/TSsd_0-225'}

# combine env data with nuisdata

create envdata and rangedata files for 01.01.MVP_gf_training_script.R

In [8]:
all_envdata = defaultdict(dict)
for nuis_envs, outerdir in outerdirs.items():
    for seed, envfile in pbar(envfiles.items(), desc=nuis_envs):
        # dir and file names to save
        gf_training_dir = makedir(op.join(outerdir, 'gradient_forests/training/training_files'))
        envfile = op.join(gf_training_dir, f'{seed}_envfile_GFready_pooled.txt')
        rangefile = envfile.replace('_envfile_', '_rangefile_')
        
        # read in envdata and nuis envs
        envdata = pd.read_table(envfiles[seed], index_col=0)
        nuisdata = pd.read_table(nuis_files[nuis_envs], index_col=0)
        
        # overwrite envdata by adding nuisdata
        envdata = pd.merge(envdata, nuisdata, left_index=True, right_index=True)
        rangedata = pd.merge(envdata, locations, left_index=True, right_index=True)  # add lat/lon to envdata
        
        # save for training
        envdata.to_csv(envfile, sep='\t', index=True)
        rangedata.to_csv(rangefile, sep='\t', index=False)
        
        # keep track for writing shfiles later
        all_envdata[nuis_envs][seed] = envfile
    
pd.read_table(envfile, index_col=0)

TSsd: 100%|███████████████| 225/225 [00:06<00:00, 32.93it/s]


,sal_opt,temp_opt,TSsd
1,-1.0,-1.0,-2.245441
2,-0.5,-1.0,-1.967663
3,0.0,-1.0,-0.963343
4,0.5,-1.0,-0.164371
5,1.0,-1.0,-0.291426
...,...,...,...
96,1.0,1.0,-0.239327
97,0.5,1.0,0.598108
98,0.0,1.0,1.669095
99,-0.5,1.0,2.462649


In [9]:
pd.read_table(rangefile)

,sal_opt,temp_opt,TSsd,lat,lon
0,-1.0,-1.0,-2.245441,1.0,1.0
1,-0.5,-1.0,-1.967663,1.0,2.0
2,0.0,-1.0,-0.963343,1.0,3.0
3,0.5,-1.0,-0.164371,1.0,4.0
4,1.0,-1.0,-0.291426,1.0,5.0
...,...,...,...,...,...
95,1.0,1.0,-0.239327,10.0,6.0
96,0.5,1.0,0.598108,10.0,7.0
97,0.0,1.0,1.669095,10.0,8.0
98,-0.5,1.0,2.462649,10.0,9.0


# symlink snp data

In [10]:
gfc_training_dir

'/home/b.lind/offsets/run_20220919_0-225/gradient_forests/training/training_files'

In [11]:
snpfiles = wrap_defaultdict(dict, 2)
for envs, outerdir in outerdirs.items():
    gf_training_dir = op.join(outerdir, 'gradient_forests/training/training_files')
    
    for seed in envfiles:
        for marker_set in ['adaptive', 'all', 'neutral']:
            # the snp data I want to use from canonical dir
            src = op.join(gfc_training_dir, f'{seed}_Rout_Gmat_sample_maf-gt-p01_GFready_pooled_{marker_set}.txt')

            # where i want to save (nuisdir)
            dst = op.join(gf_training_dir, op.basename(src))

            if not op.exists(dst):
                os.symlink(src, dst)

            snpfiles[envs][seed][marker_set] = dst
        
dst

'/home/b.lind/offsets/run_20220919_nuisance/TSsd_0-225/gradient_forests/training/training_files/1231318_Rout_Gmat_sample_maf-gt-p01_GFready_pooled_neutral.txt'

In [12]:
# sanity check
all_dsts = []
for (envs, seed, marker_set), dst in unwrap_dictionary(snpfiles):
    all_dsts.append(dst)
    
len(all_dsts), luni(all_dsts)

(4725, 4725)

# create shfiles

In [13]:
mymem = {
    'all': '300000M',
    'adaptive': '4000M',
    'neutral': '150000M'
}

In [15]:
shfiles = defaultdict(list)
for (nuis_envs, seed, marker_set), snpfile in unwrap_dictionary(snpfiles, progress_bar=True):
    # get dirs
    outerdir = outerdirs[nuis_envs]
    gf_training_dir = op.join(outerdir, 'gradient_forests/training/training_files')
    assert op.exists(gf_training_dir)
    gf_training_outdir = makedir(gf_training_dir.replace('training_files', 'training_outfiles'))
    shdir = makedir(gf_training_dir.replace('training_files', 'training_shfiles'))
    
    # identify envfile and rangefile
    envfile = all_envdata[nuis_envs][seed]
    assert '_envfile_' in envfile
    rangefile = envfile.replace("_envfile_", "_rangefile_")
    
    mem = mymem[marker_set]
    
    job = f'{seed}_{nuis_envs}_GF_training_pooled_{marker_set}'
    
    text = f'''#!/bin/bash
#SBATCH --job-name={job}
#SBATCH --time=1-00:00:00
#SBATCH --mem={mem}
#SBATCH --partition='short'
#SBATCH --output={job}_%j.out

source $HOME/.bashrc  # assumed that conda init is within .bashrc
conda deactivate
conda activate r35

cd {gf_training_dir}

/home/b.lind/anaconda3/envs/r35/lib/R/bin/Rscript \
/home/b.lind/code/MVP-offsets/01_src/MVP_gf_training_script.R \
{snpfile} \
{envfile} \
{rangefile} \
{job} \
{gf_training_outdir}
'''
    
    sh = op.join(shdir, f'{job}.sh')
    
    with open(sh, 'w') as o:
        o.write(text)
    
    shfiles[nuis_envs].append(sh)

len(shfiles), luni(shfiles)

100%|███████████████| 7/7 [00:46<00:00,  6.71s/it]


(7, 7)

In [16]:
nuis_envs

'TSsd'

In [17]:
pd.read_table(envfile, index_col=0)

,sal_opt,temp_opt,TSsd
1,-1.0,-1.0,-2.245441
2,-0.5,-1.0,-1.967663
3,0.0,-1.0,-0.963343
4,0.5,-1.0,-0.164371
5,1.0,-1.0,-0.291426
...,...,...,...
96,1.0,1.0,-0.239327
97,0.5,1.0,0.598108
98,0.0,1.0,1.669095
99,-0.5,1.0,2.462649


In [18]:
pd.read_table(rangefile)

,sal_opt,temp_opt,TSsd,lat,lon
0,-1.0,-1.0,-2.245441,1.0,1.0
1,-0.5,-1.0,-1.967663,1.0,2.0
2,0.0,-1.0,-0.963343,1.0,3.0
3,0.5,-1.0,-0.164371,1.0,4.0
4,1.0,-1.0,-0.291426,1.0,5.0
...,...,...,...,...,...
95,1.0,1.0,-0.239327,10.0,6.0
96,0.5,1.0,0.598108,10.0,7.0
97,0.0,1.0,1.669095,10.0,8.0
98,-0.5,1.0,2.462649,10.0,9.0


In [19]:
# save files since I'm not going to sbatch all of them at the same time
shpkl = op.join(maindir, 'shfiles.pkl')

pkldump(shfiles, shpkl)

shpkl

'/home/b.lind/offsets/run_20220919_nuisance/shfiles.pkl'

# sbatch in batches

because of the job names and the way I have myslurm set up, I can only sbatch files for a single outerdir (I can't have duplicate job names in the queue)

In [20]:
# overwrite object that was used in for loops above
nuis_envs = keys(nuis_files)

nuis_envs

['ISO-PSsd', 'ISO-TSsd-PSsd', 'ISO-TSsd', 'ISO', 'PSsd', 'TSsd-PSsd', 'TSsd']

In [21]:
pids = sbatch(shfiles[nuis_envs[0]])

sbatching: 100%|███████████████| 675/675 [02:35<00:00,  4.35it/s]


In [28]:
nuis_envs[0]

'ISO-PSsd'

In [23]:
Squeue(grepping=['training', '4000'], states='PD', partition='short').update(to_partition='lotterhos')

update: 100%|███████████████| 225/225 [00:20<00:00, 10.95it/s]


In [25]:
Squeue(partition='short', grepping='300000').update(
    to_partition='long', num_jobs=110
)

update: 100%|███████████████| 110/110 [00:08<00:00, 13.54it/s]


In [27]:
pids.extend(
    sbatch(shfiles[nuis_envs[1]])
)

sbatching: 100%|███████████████| 675/675 [03:12<00:00,  3.50it/s]


In [29]:
nuis_envs[1]

'ISO-TSsd-PSsd'

In [31]:
Squeue(grepping=['training', '4000'], states='PD', partition='short').update(to_partition='lotterhos')

update: 100%|███████████████| 225/225 [00:20<00:00, 11.19it/s]


In [32]:
# this time do 150000 so 300000 and 150000 are even between short and long
Squeue(partition='short', grepping='150000').update(
    to_partition='long', num_jobs=110
)

update: 100%|███████████████| 110/110 [00:10<00:00, 10.02it/s]


In [33]:
Squeue(aflag=True).partitions()

{'long': Counter({'PD': 212, 'R': 8}),
 'lotterhos': Counter({'PD': 152, 'R': 35}),
 'short': Counter({'PD': 630, 'R': 50})}

In [30]:
formatclock(dt.now() - t1)

'0-00:13:33'

In [ ]:
stop

In [ ]:
print(read(shfiles[nuis_envs[0]][0], lines=False))

In [ ]:
/home/b.lind/anaconda3/envs/r35/lib/R/bin/Rscript \
/home/b.lind/code/MVP-offsets/01_src/MVP_gf_training_script.R \
/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/gradient_forests/training/training_files/1231094_Rout_Gmat_sample_maf-gt-p01_GFready_pooled_adaptive.txt \
/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/gradient_forests/training/training_files/1231094_envfile_GFready_pooled.txt \
/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/gradient_forests/training/training_files/1231094_rangefile_GFready_pooled.txt \
1231094_GF_training_pooled_adaptive \
/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/gradient_forests/training/training_outfiles

In [ ]:
pd.read_table('/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/gradient_forests/training/training_files/1231094_envfile_GFready_pooled.txt')

# check on progress

In [1]:
from pythonimports import *

import MVP_summary_functions as mvp

mvp.latest_commit()
session_info.show()

#########################################################
Today:	April 25, 2023 - 09:14:57
python version: 3.8.5

Current commit of pythonimports:
commit 4ecd56c8c80ec4876790fcd425cb75b9db9a4f24  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Tue Mar 21 13:20:54 2023 -0400

Current commit of MVP_offsets:
commit ef86c4b94658027c580e1bbfdfc1195b0b7077e9  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Thu Mar 23 10:58:00 2023 -0400
#########################################################



In [2]:
dirs = [
    '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/gradient_forests/training/training_shfiles',
    '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/gradient_forests/training/training_shfiles'
]

In [3]:
lview, dview = get_client(cluster_id='1682428458-l8u4', profile='lotterhos')

36 36


In [4]:
seffs = {}
for d in dirs:
    nuis_envs = d.split("nuisance/")[1].split('_0-225')[0]
    
    print(nuis_envs)
    
    outs = fs(d, endswith='.out', exclude='watcher')
    
    seffs[nuis_envs] = Seffs.parallel(lview, outs=outs).most_recent()

ISO-PSsd

Watching 746 parallel Seffs jobs ...


retrieving seffs: 100%|███████████████| 746/746 [00:06<00:00, 109.56it/s]
/home/b.lind/pythonimports/myslurm.py:548: UserWarning: There are multiple shfiles associated with outfiles. len=746 luni=675
  warnings.warn(text)


ISO-TSsd-PSsd

Watching 744 parallel Seffs jobs ...


retrieving seffs: 100%|███████████████| 744/744 [00:06<00:00, 108.68it/s]
/home/b.lind/pythonimports/myslurm.py:548: UserWarning: There are multiple shfiles associated with outfiles. len=744 luni=675
  warnings.warn(text)


In [5]:
for nuis_envs, s in seffs.items():
    print(nuis_envs)
    print(s.states.counter())

ISO-PSsd
Counter({'State: COMPLETED (exit code 0)': 675})
ISO-TSsd-PSsd
Counter({'State: COMPLETED (exit code 0)': 675})


In [6]:
675 / 3  # 3 marker sets; 225 seeds

225.0

In [9]:
# save seffs
for nuis_envs, s in seffs.items():
    d = f'/home/b.lind/offsets/run_20220919_nuisance/{nuis_envs}_0-225/gradient_forests/training/training_shfiles'
    assert op.exists(d)
    
    pkl = op.join(d, 'seffs.pkl')
    
    pkldump(s, pkl)
    
    print(pkl)

/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/gradient_forests/training/training_shfiles/seffs.pkl
/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/gradient_forests/training/training_shfiles/seffs.pkl


In [ ]:
sq = Squeue(aflag=True)

In [ ]:
needed = []
for nuis_envs, s in seffs.items():
    for pid, seff in s.timeouts().items():
        if seff.job not in sq.jobs().tolist():
            needed.append(seff.sh)
            
len(needed)

In [ ]:
for sh in needed:
    text = read(sh, lines=True)
    print(text[2])
    assert 'time' in text[2]
    text[2] = '#SBATCH --time=5-00:00:00'
    
    assert 'partition' in text[4]
    text[4] = '#SBATCH --partition=long'
    
    with open(sh ,'w') as o:
        o.write('\n'.join(text))
        
sbatch(needed)

In [ ]:
shfiles = []
for nuis_envs, s in seffs.items():
    for sh in s.timeouts().shfiles:
        text = read(sh, lines=True)
        
        assert 'time' in text[2]
        text[2] = '#SBATCH --time=5-00:00:00'
        
        assert 'partition' in text[4]
        text[4] = "#SBATCH --partition=long"
        
        with open(sh, 'w') as o:
            o.write('\n'.join(text))
            
        shfiles.append(sh)
        
len(shfiles)

In [ ]:
pids = sbatch(shfiles)

In [ ]:
??sbatch

In [ ]:
for nuis_env, s in seffs.items():
    print(s.failed().states.counter())
    


In [ ]:
shfiles = []
for nuis_envs, s in seffs.items():
    for sh in s.failed().shfiles:
        text = read(sh)
        
        assert text[3] == '#SBATCH --mem=300000M'
        
        text[3] = '#SBATCH --mem=500000M'
        
        with open(sh, 'w') as o:
            o.write('\n'.join(text))
            
        shfiles.append()

In [ ]:
from pythonimports import *

latest_commit()

shpkl = '/home/b.lind/offsets/run_20220919_nuisance/shfiles.pkl'
shfiles = pklload(shpkl)

nuis_envs = ['ISO-PSsd', 'ISO-TSsd-PSsd', 'ISO-TSsd', 'ISO', 'PSsd', 'TSsd-PSsd', 'TSsd']

# pids = sbatch(shfiles[nuis_envs[2]])

In [ ]:
from pythonimports import *

latest_commit()

shpkl = '/home/b.lind/offsets/run_20220919_nuisance/shfiles.pkl'
shfiles = pklload(shpkl)

nuis_envs = ['ISO-PSsd', 'ISO-TSsd-PSsd', 'ISO-TSsd', 'ISO', 'PSsd', 'TSsd-PSsd', 'TSsd']

# pids = sbatch(shfiles[nuis_envs[3]])

In [ ]:
from pythonimports import *

latest_commit()

shpkl = '/home/b.lind/offsets/run_20220919_nuisance/shfiles.pkl'
shfiles = pklload(shpkl)

nuis_envs = ['ISO-PSsd', 'ISO-TSsd-PSsd', 'ISO-TSsd', 'ISO', 'PSsd', 'TSsd-PSsd', 'TSsd']

# pids = sbatch(shfiles[nuis_envs[4]])

In [ ]:
from pythonimports import *

latest_commit()

shpkl = '/home/b.lind/offsets/run_20220919_nuisance/shfiles.pkl'
shfiles = pklload(shpkl)

nuis_envs = ['ISO-PSsd', 'ISO-TSsd-PSsd', 'ISO-TSsd', 'ISO', 'PSsd', 'TSsd-PSsd', 'TSsd']

# pids = sbatch(shfiles[nuis_envs[5]])

In [ ]:
from pythonimports import *

latest_commit()

shpkl = '/home/b.lind/offsets/run_20220919_nuisance/shfiles.pkl'
shfiles = pklload(shpkl)

nuis_envs = ['ISO-PSsd', 'ISO-TSsd-PSsd', 'ISO-TSsd', 'ISO', 'PSsd', 'TSsd-PSsd', 'TSsd']

# pids = sbatch(shfiles[nuis_envs[6]])